In [12]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [31]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from operator import itemgetter

In [14]:
llm = ChatOpenAI(
    api_key=key, 
    model_name='gpt-4o-mini',
    temperature=0.1
)

In [15]:
prompt = PromptTemplate.from_template(
    """주어진 사용자 질문을 `수학`, `과학`, 또는 `기타` 중 하나로 분류하세요. 한 단어 이상으로 응답하지 마세요.

<question>
{question}
</question>

Classification:"""
)

In [16]:
output_parser = StrOutputParser()

In [17]:
# 체인
chain = prompt | llm | output_parser

In [18]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "2+2 는 무엇인가요?"})

'수학'

In [19]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "작용 반작용의 법칙은 무엇인가요?"})

'과학'

In [21]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "소놀코딩은 어떤 회사인가요?"})

'기타'

In [23]:
# 세 개의 하위 체인을 생성

In [24]:
math_chain = (
    PromptTemplate.from_template(
        """You are an expert in math. \
Always answer questions starting with "깨봉선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI(model="gpt-4o-mini")
)

In [25]:
science_chain = (
    PromptTemplate.from_template(
        """You are an expert in science. \
Always answer questions starting with "아이작 뉴턴 선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI(model="gpt-4o-mini")
)

In [26]:
general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question concisely:

Question: {question}
Answer:"""
    )
    | ChatOpenAI(model="gpt-4o-mini")
)

In [28]:
# 사용자 정의 함수 사용하기
# LangChain 공식 도큐먼트에서 권장하는 방법.
# 서로 다른 출력 간의 라우팅을 위해 사용자 정의 함수를 RunnableLambda 로 래핑하여 활용할 수도 있습니다.

In [30]:
def route(info):
    if "수학" in info["topic"].lower():     # 주제에 "수학"이 포함되어 있는 경우
        return math_chain                   # datascience_chain 리턴
                                            
    elif "과학" in info["topic"].lower():   # 주제에 "과학"이 포함되어 있는 경우        
        return science_chain                # art_chain 리턴
    
    else:                                   # 그 외의 경우        
        return general_chain                # general_chain 리턴

In [ ]:
full_chain = (
    {"topic": chain, "question": itemgetter("question")}
    | RunnableLambda(
        # 경로를 지정하는 함수를 인자로 전달합니다.
        route
    )
    | StrOutputParser()
)